In [76]:
from google.colab import auth, drive
from googleapiclient.discovery import build

auth.authenticate_user()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta, date
import requests

# Model

In [78]:
class CandleTransformer(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super().__init__()

        self.input_proj = nn.Linear(input_dim, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )

        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.regressor = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, 1)
        )

    def forward(self, x):
        """
        x: Tensor shape (batch_size, seq_len=6, input_dim)
        """
        x = self.input_proj(x)  # (B, 6, d_model)
        x = self.encoder(x)     # (B, 6, d_model)
        x_last = x[:, -1, :]    # (B, d_model)
        out = self.regressor(x_last)  # (B, 1)
        return out.squeeze(1)         # (B,)


# Scaler

In [79]:
class StockScaler:
    def __init__(self):
        self.stats = {}

    def fit(self, df, stock_id):
        df = df.copy()
        df['log_volume'] = np.log1p(df['volume'])

        self.stats[stock_id] = {}
        for col in ['open', 'high', 'low', 'close', 'log_volume']:
            mean = df[col].mean()
            std = df[col].std()
            self.stats[stock_id][col] = {'mean': mean, 'std': std}

    def normalize(self, df, stock_id):
        df = df.copy()
        df['log_volume'] = np.log1p(df['volume'])

        for col in ['open', 'high', 'low', 'close', 'log_volume']:
            mean = self.stats[stock_id][col]['mean']
            std = self.stats[stock_id][col]['std']
            df[col] = (df[col] - mean) / std

        return df[['open', 'high', 'low', 'close', 'log_volume']]

    def denormalize_y(self, y_pred, stock_id):
        std = self.stats[stock_id]['close']['std']
        mean = self.stats[stock_id]['close']['mean']
        return y_pred * std + mean


In [80]:
class StockScalerJSON:
    def __init__(self, stats_path):
        with open(stats_path, 'r') as f:
            self.stats = json.load(f)

    def normalize(self, df, stock_id):
        df = df.copy()
        df['log_volume'] = np.log1p(df['volume'])
        for col in ['open', 'high', 'low', 'close', 'log_volume']:
            mean = self.stats[stock_id][col]['mean']
            std = self.stats[stock_id][col]['std']
            df[col] = (df[col] - mean) / std
        return df[['open', 'high', 'low', 'close', 'log_volume']]

    def denormalize_y(self, y_pred, stock_id):
        std = self.stats[stock_id]['close']['std']
        mean = self.stats[stock_id]['close']['mean']
        return y_pred * std + mean

# Predict

In [84]:
def fetch_moex_candles(secid: str, board: str = "TQBR", interval: int = 60):
    url = (
        f"https://iss.moex.com/iss/engines/stock/markets/shares/"
        f"boards/{board}/securities/{secid}/candles.json"
    )
    params = {
        "from": date.today().isoformat(),
        "till": date.today().isoformat(),
        "interval": interval
    }
    r = requests.get(url, params=params)
    data = r.json()['candles']['data']
    cols = r.json()['candles']['columns']
    df = pd.DataFrame(data, columns=cols)
    df = df.rename(columns={
        'begin': 'datetime',
        'open': 'open',
        'high': 'high',
        'low': 'low',
        'close': 'close',
        'volume': 'volume'
    })
    df['datetime'] = pd.to_datetime(df['datetime'])
    return df[['datetime', 'open', 'high', 'low', 'close', 'volume']].tail(6)

df = fetch_moex_candles("SBER")
df.describe()

,datetime,open,high,low,close,volume
count,6,6.000000,6.000000,6.000000,6.000000,6.000000e+00
mean,2025-07-17 11:30:00,325.688333,326.951667,324.846667,326.176667,8.300598e+06
min,2025-07-17 09:00:00,323.090000,323.750000,322.880000,323.540000,2.149030e+06
25%,2025-07-17 10:15:00,324.030000,326.207500,323.782500,325.655000,5.826968e+06
50%,2025-07-17 11:30:00,325.875000,327.080000,324.760000,326.115000,6.588215e+06
75%,2025-07-17 12:45:00,327.030000,328.260000,325.842500,327.025000,9.899620e+06
max,2025-07-17 14:00:00,328.460000,329.230000,327.020000,328.460000,1.790149e+07
std,NaN,2.095952,1.953063,1.553057,1.664003,5.481132e+06


In [85]:
df

,datetime,open,high,low,close,volume
3,2025-07-17 09:00:00,323.09,323.75,322.88,323.54,2149030
4,2025-07-17 10:00:00,323.54,328.50,323.54,328.46,17901490
5,2025-07-17 11:00:00,328.46,329.23,327.02,327.29,10962250
6,2025-07-17 12:00:00,327.29,327.54,326.12,326.23,6711730
7,2025-07-17 13:00:00,326.25,326.62,325.01,325.54,6464700
8,2025-07-17 14:00:00,325.50,326.07,324.51,326.00,5614390


In [87]:
scaler = StockScaler()
scaler.fit(df, 'SBER')

df_normalized = scaler.normalize(df, 'SBER')
x = torch.tensor(df_normalized.values, dtype=torch.float32).unsqueeze(0)
x

tensor([[[-1.2397, -1.6393, -1.2663, -1.5845, -1.6280],
         [-1.0250,  0.7928, -0.8414,  1.3722,  1.3515],
         [ 1.3224,  1.1665,  1.3994,  0.6691,  0.6622],
         [ 0.7642,  0.3012,  0.8199,  0.0321, -0.0274],
         [ 0.2680, -0.1698,  0.1052, -0.3826, -0.0801],
         [-0.0899, -0.4514, -0.2168, -0.1062, -0.2783]]])

In [88]:
scalerJson = StockScalerJSON('/content/drive/MyDrive/Colab Notebooks/scaler_stats.json')

df_normalized2 = scalerJson.normalize(df, 'SBER')
x2 = torch.tensor(df_normalized2.values, dtype=torch.float32).unsqueeze(0)
x2

tensor([[[2.3218, 2.3175, 2.3316, 2.3270, 0.9639],
         [2.3267, 2.3688, 2.3388, 2.3803, 1.6524],
         [2.3800, 2.3767, 2.3766, 2.3677, 1.4932],
         [2.3673, 2.3585, 2.3668, 2.3562, 1.3338],
         [2.3561, 2.3485, 2.3547, 2.3487, 1.3216],
         [2.3479, 2.3426, 2.3493, 2.3537, 1.2758]]])

In [89]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CandleTransformer(input_dim=5)
model.load_state_dict(torch.load('/content/drive/MyDrive/weights/best_transformer_v2.pth', map_location='cpu'))

model.to(device)
x = x.to(device)

model.eval()
with torch.no_grad():
    output = model(x)

print(output.item())
print(scaler.denormalize_y(output.item(), 'SBER'))
print(scaler.stats)

-0.16226032376289368
325.9066649678606
{'SBER': {'open': {'mean': np.float64(325.68833333333333), 'std': 2.0959524485700207}, 'high': {'mean': np.float64(326.95166666666665), 'std': 1.9530634056954448}, 'low': {'mean': np.float64(324.84666666666664), 'std': 1.5530572000627172}, 'close': {'mean': np.float64(326.1766666666667), 'std': 1.6640032051251075}, 'log_volume': {'mean': np.float64(15.738828135933487), 'std': 0.7114863572757094}}}


In [90]:
x2 = x2.to(device)

model.eval()
with torch.no_grad():
    output = model(x2)

print(output.item())
print(scalerJson.denormalize_y(output.item(), 'SBER'))

2.3470458984375
325.38722280406995


close свечи с открытием в 14.00 был 326.00